In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install datasets

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
torchctr_root = '../'

import sys
sys.path.append(torchctr_root)

In [4]:
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim import Adam

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Load data

In [7]:
import joblib

df_samples = joblib.load(f'{torchctr_root}/data/amazon_data.pkl')

In [8]:
len( df_samples['reviewerID'].unique() )

192403

In [9]:
len( df_samples['asin'].unique() )

63001

In [10]:
len( df_samples['brand'].unique() )

3526

In [11]:
len( df_samples['categories'].unique() )

801

In [12]:
# sequence features from latest to oldest
df_samples['his_asin_seq'] = df_samples['his_asin_seq'].map(lambda x: x[::-1])

## Create pytorch dataset

In [72]:
## Hash buckets
# feat_configs = [
#     {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
#     {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
    
#     {"name": "price", "dtype": "numerical", "norm": "std"},
#     {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
#     {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

#     {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12, "hash_buckets": 1000000},
# ]

## Dynamic Embedding
# feat_configs = [
#     {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3},
#     {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3},
    
#     {"name": "price", "dtype": "numerical", "norm": "std"},
#     {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
#     {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

#     {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12},
# ]

## Auto generate feat_configs
from torchctr.utils import auto_generate_feature_configs
feat_configs = auto_generate_feature_configs(
    df_samples[['reviewerID', 'asin', 'price', 'brand', 'categories', 'his_asin_seq']]
)

print(feat_configs)

target_cols = ['label', ]

[{'name': 'reviewerID', 'dtype': 'category', 'emb_dim': 17, 'min_freq': 3}, {'name': 'asin', 'dtype': 'category', 'emb_dim': 15, 'min_freq': 3}, {'name': 'price', 'dtype': 'numerical', 'norm': 'std', 'mean': np.float64(74.40153304932919), 'std': np.float64(123.75264929565961)}, {'name': 'brand', 'dtype': 'category', 'emb_dim': 11, 'min_freq': 3}, {'name': 'categories', 'dtype': 'category', 'emb_dim': 9, 'min_freq': 3}, {'name': 'his_asin_seq', 'dtype': 'category', 'islist': True, 'emb_dim': 15, 'min_freq': 3, 'max_len': 256}]


In [86]:
from torchctr.sample import traintest_split

df_train, df_test = traintest_split(df_samples, test_size=0.2, shuffle=True, group_id='reviewerID')
print(len(df_train), len(df_test))

1352538 336650


In [87]:
# from torchctr.dataset import FeatureTransformer

# transformer = FeatureTransformer(feat_configs, category_dynamic_vocab=True, category_min_freq=3, verbose=True)
# transformer.fit(df_train)
# df_train = transformer.transform(df_train)
# df_test = transformer.transform(df_test)

In [88]:
# to pl.DataFrame
import polars as pl

df_train = pl.from_pandas(df_train)
df_test = pl.from_pandas(df_test)

In [89]:
from torchctr.dataset import FeatureTransformer

transformer = FeatureTransformer(feat_configs, category_dynamic_vocab=True, category_min_freq=3, verbose=True)

df_train = transformer.fit_transform(df_train, is_train=True, n_jobs=4)
df_test = transformer.transform(df_test, n_jobs=4)

2024-11-11 21:40:09 torchctr INFO - Feature transforming (is_train=True), note that feat_configs will be updated when is_train=True...
2024-11-11 21:40:09 torchctr INFO - Input dataFrame type: <class 'polars.dataframe.frame.DataFrame'>, transform it by FeatureTransformer


2024-11-11 21:40:14 torchctr INFO - Processing feature reviewerID...
2024-11-11 21:40:14 torchctr INFO - Processing feature price...
2024-11-11 21:40:14 torchctr INFO - Feature price mean: 74.40153304932919, std: 123.75264929565961, min: 0.01, max: 999.99
2024-11-11 21:40:15 torchctr INFO - Processing feature asin...
2024-11-11 21:40:15 torchctr INFO - Processing feature categories...
2024-11-11 21:40:15 torchctr INFO - Processing feature brand...
2024-11-11 21:40:16 torchctr INFO - Converting category reviewerID to indices...
2024-11-11 21:40:16 torchctr INFO - Converting category asin to indices...
2024-11-11 21:40:16 torchctr INFO - Converting category brand to indices...
2024-11-11 21:40:16 torchctr INFO - Feature brand vocab size: None -> 3503
2024-11-11 21:40:16 torchctr INFO - Feature asin vocab size: None -> 62384
2024-11-11 21:40:16 torchctr INFO - Feature reviewerID vocab size: None -> 153923
2024-11-11 21:40:16 torchctr INFO - Converting category categories to indices...
202

In [90]:
df_test.head()

reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label,his_asin_seq
i32,i32,i64,f64,str,f64,i32,i32,i64,list[i32]
0,60021,1386547200,4.0,"""Mediabridge 16AWG Speaker Wire…",-0.480083,217,568,1,"[41675, 5429]"
0,61124,1362268800,5.0,"""Monoprice 107165 3.5mm Stereo …",-0.583757,1929,459,1,[59789]
0,56065,1355270400,5.0,"""Poetic (TM) PU Folio Case for …",-0.601131,1223,576,1,[6154]
0,59007,1210204800,5.0,"""Sony STRDG720 7.1 Audio Video …",1.984188,1223,607,1,"[4982, 61032, … 15221]"
0,56272,1387670400,5.0,"""Corsair Vengeance 16GB (2x8GB…",0.891847,750,425,1,[43254]


In [91]:
from datasets import Dataset

# train_dataset = Dataset.from_pandas(df_train).with_format('torch', columns=[x['name'] for x in feat_configs] + target_cols)
# test_dataset = Dataset.from_pandas(df_test).with_format('torch', columns=[x['name'] for x in feat_configs] + target_cols)

train_dataset = Dataset.from_polars(df_train).with_format('torch', columns=[x['name'] for x in feat_configs] + target_cols)
test_dataset = Dataset.from_polars(df_test).with_format('torch', columns=[x['name'] for x in feat_configs] + target_cols)

In [92]:
from torchctr.functional import pad_sequences_to_maxlen

def collate_fn(batch):
    # list of dict to dict of list
    # print(batch)

    batch_dense = []
    batch_sparse = {}
    for k in transformer.feat_configs:
        if k['type'] == 'dense':
            # print(k['name'])
            batch_dense.append(
                torch.tensor([sample[k['name']] for sample in batch], dtype=torch.float32)
            )
        elif k['type'] == 'sparse':
            if k.get('islist'):
                sparse_feat = [sample[k['name']] for sample in batch]
                # pad sequences
                sparse_feat = pad_sequences_to_maxlen(sparse_feat, batch_first=True, padding_value=-100, max_length=128)
            else:
                sparse_feat = torch.tensor([[sample[k['name']]] for sample in batch], dtype=torch.long)
            batch_sparse[k['name']] = sparse_feat

    batch_features = {
        'dense_features': torch.stack(batch_dense, dim=1),
        **batch_sparse
    }
    batch_labels = torch.tensor([[sample['label']] for sample in batch], dtype=torch.float32)
    
    return batch_features, batch_labels

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False, collate_fn=collate_fn)

In [93]:
print( len(train_dataloader) )
for features, labels in DataLoader(train_dataset, batch_size=2, num_workers=0, shuffle=True, collate_fn=collate_fn):
    print(features)
    print(labels)
    break

2642
{'dense_features': tensor([[-0.1165],
        [-0.4276]]), 'reviewerID': tensor([[148180],
        [124848]]), 'asin': tensor([[31412],
        [47720]]), 'brand': tensor([[1223],
        [3027]]), 'categories': tensor([[302],
        [557]]), 'his_asin_seq': tensor([[31468, 61039,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  

## Train Model

In [64]:
from torchctr.models import DNN

dnn_hidden_units = [128,64,32]
model = DNN(transformer.feat_configs, hidden_units=dnn_hidden_units)
model = model.to(device)
print(model)

2024-11-11 15:24:05 torchctr INFO - Model Input: dense_size=1, sparse_size=67


DNN(
  (embeddings): ModuleDict(
    (reviewerID): Embedding(153923, 17)
    (asin): Embedding(62384, 15)
    (brand): Embedding(3504, 11)
    (categories): Embedding(800, 9)
    (his_asin_seq): Embedding(61926, 15)
  )
  (tower): Sequential(
    (0): Linear(in_features=68, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [65]:
optimizer = Adam(model.parameters(),  lr = 0.002, weight_decay = 1e-9)
lr_scd = lr_scheduler.StepLR(optimizer, step_size=len(train_dataloader), gamma=0.8)

In [66]:
from torchctr.trainer import Trainer

trainer = Trainer(
    model, 
    optimizer=optimizer,
    lr_scheduler=lr_scd,
    max_epochs=5,
    early_stopping_rounds=3,
    save_ckpt_path='./ckpt/'
)

model = trainer.fit(train_dataloader, eval_dataloader = test_dataloader, ret_model = 'final') #, init_ckpt_path='./ckpt/')

2024-11-11 15:24:45 torchctr INFO - [Validation] Epoch: 0/5, Validation Loss: {'loss': 0.6933948981001022}
2024-11-11 15:24:45 torchctr INFO - Learning rate: 0.002
2024-11-11 15:24:45 torchctr INFO - [Training] Epoch: 1/5 iter 0/2642, Training Loss: {'loss': 0.6966052651405334}
2024-11-11 15:24:55 torchctr INFO - [Training] Epoch: 1/5 iter 100/2642, Training Loss: {'loss': 0.4531710368394852}
2024-11-11 15:25:04 torchctr INFO - [Training] Epoch: 1/5 iter 200/2642, Training Loss: {'loss': 0.4170465312898159}
2024-11-11 15:25:13 torchctr INFO - [Training] Epoch: 1/5 iter 300/2642, Training Loss: {'loss': 0.4033564534783363}
2024-11-11 15:25:21 torchctr INFO - [Training] Epoch: 1/5 iter 400/2642, Training Loss: {'loss': 0.39582929089665414}
2024-11-11 15:25:31 torchctr INFO - [Training] Epoch: 1/5 iter 500/2642, Training Loss: {'loss': 0.3896539005637169}
2024-11-11 15:25:41 torchctr INFO - [Training] Epoch: 1/5 iter 600/2642, Training Loss: {'loss': 0.3855347283184528}
2024-11-11 15:25:5

In [67]:
ckpt = trainer.load_ckpt('./ckpt')
model.load_state_dict(ckpt['model'].state_dict())

/mnt/wsl/dev/torchctr/examples/../torchctr/trainer.py:282: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_file)
2024-11-11 15:39:40 torchctr INFO - Loa

<All keys matched successfully>

In [68]:
test_preds = []
test_labels = []
model.eval()

for features, labels in test_dataloader:
    outputs = model(features)
    test_preds.append(outputs[:,0])
    test_labels.append(labels[:,0])
test_preds = torch.concat(test_preds, dim=0).detach().cpu().numpy()
test_labels = torch.concat(test_labels, dim=0).detach().cpu().numpy()

In [69]:
print(test_preds.shape, test_labels.shape)

(336650,) (336650,)


In [70]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(test_labels, test_preds)
print("AUC Score:", auc_score)

AUC Score: 0.6742899835345096
